In [59]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import hvplot.pandas
import holoviews as hv

In [60]:
#Import data for nn_continuous value prediction
def import_data_nn_predict(sector, select_ticker):
    #EH:  Create DataFrame from csv
    sector_data_df = pd.read_csv(Path(f'Data_Prep_Output/{sector}.csv'))
    sector_data_df=sector_data_df[sector_data_df['ticker']==select_ticker]
    actual_target_df=sector_data_df.loc[:,['date_x','q_roi']]
    actual_target_df=actual_target_df.reset_index()
    #EH:  Drop unnamed, currency, ticker, sector columns from the DataFrame
    sector_data_df = sector_data_df.drop(columns=['Unnamed: 0','reportedCurrency','ticker','Sector','date_x','q_roi'],axis=1)
    
    #EH: update int to float
    int_list=list(sector_data_df.dtypes[sector_data_df.dtypes == "int64"].index)
    sector_data_df[int_list]=sector_data_df[int_list].astype('float64')    
    
    # return data to predict, actual q_roi
    return sector_data_df, actual_target_df 

In [61]:
def scale(df):
    #Create the scaler instance
    X_scaler=StandardScaler()
    
    #Fit the scaler
    X_scaler.fit(df)
    
    #scale the features data
    X_scaled=X_scaler.transform(df)
    
    return X_scaled

In [62]:
def nn_predict(sector,select_ticker):
    
    #EH: set path of nn predict file
    model_path=Path(f'Models/nn_{sector}.h5')
    
    #EH: load the model to new object
    neuron=tf.keras.models.load_model(model_path)
    
    #EH:  create df for predict and actual roi
    ticker_feature, ticker_actual_roi=import_data_nn_predict(sector,select_ticker)
    
    #EH:  standard scale data
    ticker_feature_scaled=scale(ticker_feature)
    
    #EH: adjust data to array
    ticker_feature_scaled=np.asarray(ticker_feature_scaled).astype('float32')
    
    #EH:  make predictions
    predictions=(neuron.predict(ticker_feature_scaled))
    
    #EH: create a df to compare predictions with actual roi
    results = pd.DataFrame({"predictions": predictions.ravel()})
    # results=
    results = pd.concat([ticker_actual_roi,results],axis=1)
    
    #EH: update column to datetime
    results['date_x']=pd.to_datetime(results['date_x'])

    
    #EH: Sort date
    results=results.sort_values(by=['date_x'])
    
    #EH: set index
    results.set_index('date_x',inplace=True)
    
    #EH: Calculate cumulative quarter return
    results['q_roi cumulative retun']=((results['q_roi']+1).cumprod())-1
    results['predict cumulative retun']=((results['predictions']+1).cumprod())-1
    
    #EH: drop extra column
    results.drop(columns=['index'],inplace=True)
    
    #EH:  plot
    results_plot=results.loc[:,['q_roi cumulative retun','predict cumulative retun']].hvplot(ylabel='%',title=f'{select_ticker} Quarterly cumulative return % - Neural Network Model')
    
    return results, results_plot
    
    
    
    
    

In [93]:
results, results_plot=nn_predict('Electronic Technology','INTC')

In [94]:
results

,q_roi,predictions,q_roi cumulative retun,predict cumulative retun
date_x,,,,
2017-06-30,0.004973,0.061576,0.004973,0.061576
2017-09-30,-0.228715,-0.091087,-0.224879,-0.035120
2017-12-31,0.112558,-0.013070,-0.137633,-0.047731
2018-03-31,-0.000933,-0.218046,-0.138438,-0.255369
2018-06-30,0.061964,0.001692,-0.085052,-0.254109
2018-09-30,0.043642,-0.013295,-0.045122,-0.264026
2018-12-31,-0.286743,0.055808,-0.318926,-0.222953
2019-03-31,0.272415,-0.140084,-0.133392,-0.331805
2019-06-30,0.141266,0.040574,-0.010970,-0.304694


In [95]:
results_plot

:NdOverlay   [Variable]
   :Curve   [date_x]   (value)